# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [23]:
boston_features.head()
X = boston_features[['CHAS', 'RM', 'B', 'DIS', 'LSTAT']]
y = pd.DataFrame(boston.target, columns=['target'])


,target
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


## Train test split

Perform a train-test-split with a test set of 0.20.

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [18]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [20]:
from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train MSE: ', train_mse, '      Test MSE: ', test_mse)

Train MSE:  21.498439818357134       Test MSE:  22.805154627320913


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger than later ones.

We want the folds to be a list of subsets of data!

In [26]:
def kfolds(data, k):
    # Force data as pandas dataframe
    data = pd.DataFrame(data)
    
    # Determine lengths of dataset, folds, and remainder
    data_length = len(data)
    fold_length = data_length // k
    remainder = data_length % k
    
    # Initialize empty cut_point and folds list
    cut_point = 0
    folds = []
    
    # Loop through the k folds
    for fold_number in range(1, k+1):
        
        # Add 1 to the first few folds to use up any remainder
        if fold_number <= remainder:
            fold = data.iloc[cut_point:cut_point + fold_length + 1]
            folds.append(fold)
            cut_point += fold_length + 1
        
        # Otherwise, just slice the data
        else: 
            fold = data.iloc[cut_point:cut_point + fold_length]
            folds.append(fold)
            cut_point += fold_length
    
    # Return the list of folds
    return folds

### Apply it to the Boston Housing Data

In [27]:
# Make sure to concatenate the data again

In [30]:
data = pd.concat([X.reset_index(drop=True), y], axis=1)
boston_folds = kfolds(data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [33]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for index, fold in enumerate(boston_folds) if index != n])
    test =  boston_folds[n]
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_resid = y_hat_train - train[y.columns]
    test_resid = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_resid.astype(float)**2))
    test_errs.append(np.mean(test_resid.astype(float)**2))
    
    
print(train_errs)
print(test_errs)


[target    24.195577
dtype: float64, target    23.032087
dtype: float64, target    19.745073
dtype: float64, target    15.317101
dtype: float64, target    22.329973
dtype: float64]
[target    13.405145
dtype: float64, target    17.444017
dtype: float64, target    37.032711
dtype: float64, target    58.279544
dtype: float64, target    26.097989
dtype: float64]


In [37]:
print('Mean train errors: ', np.mean(train_errs))
print('Mean test errors: ', np.mean(test_errs))

Mean train errors:  20.92396235372867
Mean test errors:  30.45188114354034


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [34]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

five_mses = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')
five_mses

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [36]:
mean_mse = np.mean(five_mses)
mean_mse

-30.451881143540312

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!